From explore.ipynb we can find the images that we can augment by rotating, flipping or mirroring.
## TODO

Ideas:

- Balance data by augmentation (Create more new pictures for underrepresented signs)
- Assign different signs to different augmentation methods --> What is possible for which method?

Signs:

https://user-images.githubusercontent.com/20029691/27014682-c4bcd8ca-4efd-11e7-8360-a3489b9e22c2.png

- 1) Speed Limit Zone beginning (30) --> Ordner: 00001
- 2) Speed Limit Zone end (30) --> Ordner: KEINE DATEN bis jetzt
- 3) Crosswalk --> Ordner: KEINE DATEN bis jetzt
- 4) Barred area --> Ordner: KEINE DATEN bis jetzt
- 5) Expressway beginning --> Ordner: KEINE DATEN bis jetzt
- 6) Expressway end --> Ordner: KEINE DATEN bis jetzt
- 7) Intersections STOP --> Ordner: 00014
- 8) Intersections "Vorfahrt gewähren" --> Ordner: 00013
- 9) Intersections "Vorfahrtsstraße"  --> Ordner: 00012
- 10) Turn Left  --> Ordner: 00034
- 11) Turn Right --> Do we need the counterparts here?  --> Ordner: 00033
- 12) No-passing zone beginning --> Ordner: 00009
- 13) No-passing zone end  --> Ordner: 00041
- 14) Sharp turn left small --> Ordner: KEINE DATEN bis jetzt
- 15) Sharp turn left large --> Ordner: KEINE DATEN bis jetzt
- 16) Sharp turn right small --> Do we need the counterparts here? --> Ordner: KEINE DATEN bis jetzt
- 17) Sharp turn right large --> Do we need the counterparts here? --> Ordner: KEINE DATEN bis jetzt
- 18) Steep hill uphill grade --> Ordner: KEINE DATEN bis jetzt
- 19) Steep hill downhill grade --> Ordner: KEINE DATEN bis jetzt

Methods to augment data + for which signs applicable:

-Flipping (horizontal/vertical)
    
    --> check possible invariances (e.g turn left signs) 
        and signs which can then be detected as other signs
        
    Horizontal:
    9)
    
    Vertical:
    5), 8), 9), 10) gets 11), 11) gets 10), 16) gets 17), 17) gets 16)
    
    Horizontal + Vertical:
    9)

-Rotating




## TODO

In [ ]:
fippable_horizontally = [1, 5]

In [ ]:
#Dict list for signs ('name': 'folderdirectory')
dict1 = {'Speed Limit Zone beginning (30)': '/00001', 'Speed Limit Zone end (30)': '-', 'Crosswalk': '-', 'Barred area': '-', 'Expressway beginning': '-', 'Expressway end': '-', 'Intersections STOP': '/00014', 'Intersections "Vorfahrt gewähren"': '/00013', 'Intersections "Vorfahrtsstraße"': '/00012', 'Turn Left': '/00034', 'Turn Right': '/00033', 'No-passing zone beginning': '/00009', 'No-passing zone end': '/00041', 'Sharp turn left small': '-', 'Sharp turn left large': '-', 'Sharp turn right small': '-', 'Sharp turn right large': '-', 'Steep hill uphill grade': '-', 'Steep hill downhill grade': '-'}

#Print all sign types + link to the folders
for el in dict1:
    print (el, "-->", dict1[el], )

In [ ]:
#Count images per cell
from gtsrb_loader.get_folderpath import get_folderpath
import os, os.path
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

path=get_folderpath('train');

number_list = [0] #element 0 is 0
sign_list = ['a']

i = 0
for k in iter(dict1.keys()):
    if dict1[k] == '-':
        number_files = 0
    else:
        changed_path=path+dict1[k]
        list = os.listdir(changed_path) # dir is your directory path
        number_files = len(list)
    number_list.append(number_files)
    sign_list.append(k)

#Histogram
plt.bar(range(len(number_list)), number_list)
plt.xticks(range(1, len(number_list)))
plt.title("Number of Files per folder")
plt.xlabel("Signs")
plt.ylabel("Number of Files")
plt.show()

#Legende
for i in range(1, len(number_list)):
    print (str(i) + ": " + sign_list[i] + " --> " + str(number_list[i]))

In [ ]:
import matplotlib.pyplot as plt
from gtsrb_loader._download_gtsrb import download_gtsrb

from skimage import data
from skimage.transform import resize
from PIL import Image

import numpy as np
import pandas as pd

import sys
import os
import fnmatch

In [ ]:
absolute_path = download_gtsrb()

In [ ]:
ppm_filelist = []
csv_filelist = []

for root, dirs, files in os.walk(absolute_path):
    ppm_filelist += [os.path.join(root, file) for file in files if fnmatch.fnmatch(file, '*.ppm')]
    csv_filelist += [os.path.join(root, file) for file in files if fnmatch.fnmatch(file, '*.csv')]

In [ ]:
print(ppm_filelist[0])
print(os.path.split(ppm_filelist[0])[1])
print (os.path.dirname(ppm_filelist[0]))
directory =os.path.dirname(ppm_filelist[0])+'M'
print (directory)

In [ ]:
def mirrorVer(pic) :
    v=pic.shape[1]
    i=0
    picnew=np.zeros((pic.shape[0],pic.shape[1],pic.shape[2]))
    while i < v :
        picnew[:,i,:]=pic[:,v-i-1,:]
        i=i+1
    return picnew

In [ ]:
def mirrorHor(pic) :
    h=pic.shape[0]
    i=0
    picnew=np.zeros((pic.shape[0],pic.shape[1],pic.shape[2]))
    while i < h :
        picnew[i,:,:]=pic[h-i-1,:,:]
        i=i+1
    return picnew

In [ ]:
def mirrorImages (ppm_filelist):
    for idx, filepath in enumerate(ppm_filelist):
        p=os.path.dirname(filepath)
        vertical=['00013','00012','00034','00033']
        horrizontal=['00012']
        vertAndHorr=['00012']
        if any(v in p for v in vertical):
            pic = plt.imread(filepath)
            newpic=mirrorVer(np.asarray(pic))
            directory =p+'Mv'
            im = Image.fromarray(np.uint8(newpic))
            if not os.path.exists(directory):
                os.makedirs(directory)
            im.save(directory+'\\'+os.path.split(filepath)[1])

        if any(h in p for h in horrizontal):
            pic = plt.imread(filepath)
            newpic=mirrorHor(np.asarray(pic))
            directory =p+'Mh'
            im = Image.fromarray(np.uint8(newpic))
            if not os.path.exists(directory):
                os.makedirs(directory) 
            im.save(directory+'\\'+os.path.split(filepath)[1])
            
        if any(vh in p for vh in vertAndHorr):
            pic = plt.imread(filepath)
            newpic=mirrorVer(np.asarray(pic))
            newpic=mirrorHor(newpic)
            directory =p+'Mhv'
            im = Image.fromarray(np.uint8(newpic))
            if not os.path.exists(directory):
                os.makedirs(directory) 
            im.save(directory+'\\'+os.path.split(filepath)[1])   

In [ ]:
def transform_csv_vert(df: pd.DataFrame) -> pd.DataFrame:
    columns=['Width', 'Height', 'Roi.X1', 'Roi.Y1', 'Roi.X2', 'Roi.Y2', 'ClassId']
    matrix = df.as_matrix(columns=columns)
    matrix.shape
    matrix2 = np.zeros(matrix.shape)
    for i, row in enumerate(matrix):
        matrix2[i] =[row[0],row[1],row[0]-row[2],row[3],row[0]-row[4],row[5], row[6]]
    matrix2 = matrix2.astype(int)
    df2 = pd.DataFrame(matrix2, columns=columns)
    df2['Filename'] = df['Filename']
    df2 = df2[['Filename'] + columns]
    return df2

In [ ]:
def transform_csv_horr(df: pd.DataFrame) -> pd.DataFrame:
    columns=['Width', 'Height', 'Roi.X1', 'Roi.Y1', 'Roi.X2', 'Roi.Y2', 'ClassId']
    matrix = df.as_matrix(columns=columns)
    matrix.shape
    matrix2 = np.zeros(matrix.shape)
    for i, row in enumerate(matrix):
        matrix2[i] =[row[0],row[1],row[2],row[1]-row[3],row[4],row[1]-row[5], row[6]]
    matrix2 = matrix2.astype(int)
    df2 = pd.DataFrame(matrix2, columns=columns)
    df2['Filename'] = df['Filename']
    df2 = df2[['Filename'] + columns]
    return df2

In [ ]:
def mirrorBBox (csv_filelist):
    for filepath in csv_filelist:
        df = pd.read_csv(filepath, sep=';')
        
        p=os.path.dirname(filepath)
        vertical=['00013','00012','00034','00033']
        horrizontal=['00012']
        vertAndHorr=['00012']
        if any(v in p for v in vertical):
            df2 = transform_csv_vert(df)
            directory =p+'Mv'
            if not os.path.exists(directory):
                os.makedirs(directory)
            df2.to_csv(directory+'\\'+os.path.split(filepath)[1], sep=';', index=False)

        if any(h in p for h in horrizontal):
            df2 = transform_csv_horr(df)
            directory =p+'Mh'
            if not os.path.exists(directory):
                os.makedirs(directory) 
            df2.to_csv(directory+'\\'+os.path.split(filepath)[1], sep=';', index=False)
            
        if any(vh in p for vh in vertAndHorr):
            df2 = transform_csv_vert(df)
            df2 = transform_csv_horr(df2)
            directory =p+'Mhv'
            if not os.path.exists(directory):
                os.makedirs(directory) 
            df2.to_csv(directory+'\\'+os.path.split(filepath)[1], sep=';', index=False)

In [ ]:
mirrorImages(ppm_filelist)
mirrorBBox(csv_filelist)